# Lambda Iterations

In [1]:
import os
import sys

magritteFolder = '/home/frederik/Dropbox/GitHub/Magritte/'

sys.path.append(magritteFolder)

import numpy             as np
import matplotlib.pyplot as plt
import magritte.setup    as setup
import magritte.core     as magritte

In [2]:
dimension = 1
npoints   = 100
nrays     = 200
nspecs    = 5
nlspecs   = 1
nquads    = 15

r_in   = 1.0E13   # [m]
r_out  = 7.8E16   # [m]
nH2_in = 2.0E13   # [m^-3]
X_mol  = 1.0E-8   # [.]
temp   = 20.0     # [K]
turb   = 150.00   # [.]


def nH2 (r):
    return nH2_in * np.power(r_in/r, 2.0)

def nTT (r):
    return X_mol  * nH2(r)


rs = np.logspace (np.log10(r_in), np.log10(r_out), npoints, endpoint=True)


modelFile = 'vanZadelhoff_1a_1D.hdf5'
lamdaFile = f'{magritteFolder}tests/data/test.txt'


model = magritte.Model ()
model.parameters.set_spherical_symmetry(True)
model.parameters.set_pop_prec          (1.0e-6)
model.parameters.set_model_name        (modelFile)
model.parameters.set_dimension         (dimension)
model.parameters.set_npoints           (npoints)
model.parameters.set_nrays             (nrays)
model.parameters.set_nspecs            (nspecs)
model.parameters.set_nlspecs           (nlspecs)
model.parameters.set_nquads            (nquads)

model.geometry.points.position.set([[r, 0, 0] for r in rs])
model.geometry.points.velocity.set(np.zeros((npoints, 3)))

model.chemistry.species.abundance = [[     0.0, nTT(r), nH2(r),  0.0,      1.0] for r in rs]
model.chemistry.species.symbol    =  ['dummy0', 'test',   'H2', 'e-', 'dummy1']

model.thermodynamics.temperature.gas  .set( temp                 * np.ones(npoints))
model.thermodynamics.turbulence.vturb2.set((turb/magritte.CC)**2 * np.ones(npoints))

model = setup.set_Delaunay_neighbor_lists (model)
model = setup.set_Delaunay_boundary       (model)
model = setup.set_boundary_condition_CMB  (model)
model = setup.set_rays_spherical_symmetry (model)
model = setup.set_linedata_from_LAMDA_file(model, lamdaFile)
model = setup.set_quadrature              (model)

model.write()
model.read ()

model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

['cmb', 'cmb']


0

In [3]:
def max_relative_difference(a, b):
    return np.max(2.0 * np.abs((a - b) / (a + b)))

In [4]:
Nmax = 100

pops = []
pop  = np.copy(model.lines.lineProducingSpecies[0].population)
pops.append(pop)

for it in range(Nmax):
    # Compute radiation field and resulting populations
    model.compute_radiation_field()
    model.compute_Jeff()
    model.compute_level_populations_from_stateq()
    # Compute the maximum relative change
    pop = np.copy(model.lines.lineProducingSpecies[0].population)
    pops.append(pop)
    max_diff = max_relative_difference(pops[-1], pops[-2])
    # Check for convergence
    if (max_diff < 1.0e-6):
        break
    print('After', it, 'iteration(s), max_diff =', max_diff)

After 0 iteration(s), max_diff = 1.396515213594935357
After 1 iteration(s), max_diff = 0.5309831251917840938
After 2 iteration(s), max_diff = 0.32563462102252937967
After 3 iteration(s), max_diff = 0.23137569522567772061
After 4 iteration(s), max_diff = 0.16983752740820533581
After 5 iteration(s), max_diff = 0.122350813357215412254
After 6 iteration(s), max_diff = 0.08486047808066352476
After 7 iteration(s), max_diff = 0.05674414669651036924
After 8 iteration(s), max_diff = 0.036741237378306990357
After 9 iteration(s), max_diff = 0.023311967088741446318
After 10 iteration(s), max_diff = 0.014542755176419202352
After 11 iteration(s), max_diff = 0.008988290086022146678
After 12 iteration(s), max_diff = 0.0055359174317171741
After 13 iteration(s), max_diff = 0.0033965158228948814486
After 14 iteration(s), max_diff = 0.0020795938982373599642
After 15 iteration(s), max_diff = 0.0012719862327442149582
After 16 iteration(s), max_diff = 0.0007776935054408728275
After 17 iteration(s), max_diff 